# UNGDC
## Data collection and preparation
### Unstructured Data Analytics, Spring 2020 <br>

Laura Goyeneche  <br>
Lindsay Graff    <br>
Jana Macickova   <br>
Ashita Vadlamudi <br>

Heinz College of Information Systems and Public Policy <br>
Carnegie Mellon University

In [1]:
# Libraries 
import glob
import numpy as np
import pandas as pd
from ipynb.fs.full.uda_functions import makeWordList, read_docs

In [2]:
# Import UNGDC statements
files   = [i for i in glob.glob('./data_sessions/*/*.txt')]
files71 = [i for i in glob.glob('./data_sessions/Session 71 - 2016/*')]
docs    = [read_docs(i) for i in files] + [read_docs(i) for i in files71]

In [3]:
# Create year and country list for each statemets
year    = [int(i.split('\\')[2][7:-4].strip('_')) for i in files] + [int(i.split('\\')[1][-4:]) for i in files71]
country = [i.split('\\')[2][:3] for i in files] + [i.split('\\')[1][:3] for i in files71]

In [4]:
# Replace year for cases with different format
for i,j in enumerate(year):
    if j == 11: year[i] = 2011
    if j == 12: year[i] = 2012
    if j == 13: year[i] = 2013
    if j == 14: year[i] = 2014
    if j == 15: year[i] = 2015

In [5]:
# Upload additional data sources
undocs   = pd.DataFrame({'text': docs, 'year': year, 'countryID': country})
codebook = pd.read_csv('./data_sessions/speakers_by_session.csv', sep = ',', header = 0)
wbank    = pd.read_csv('./data_worldbank/ESGData.csv', sep = ',', header = 0)
wbankdet = pd.read_csv('./data_worldbank/ESGCountry_country_details.csv', sep = ',', header = 0)

In [6]:
# Rename columns
# Guarantee that common keys have same name
codebook = codebook.rename(columns = {'Year':'year','ISO Code': 'countryID'})
wbank    = wbank.rename(columns = {"Country Code": "countryID", "Indicator Code": "indicator"})
wbankdet = wbankdet[['Country Code','Region','Income Group']]
wbankdet = wbankdet.rename(columns = {'Country Code':'countryID','Region':'region','Income Group':'income_level'})

In [7]:
# Reshape world bank indicators
wbank2   = pd.wide_to_long(wbank, ["y"], i = ["countryID","indicator"], j = "year").reset_index()
wbank2   = wbank2.rename(columns = {'y': 'indicator_val'}) 

In [8]:
# Create one datatable
# Merge Un docs, world BAnk Indicators and World Bank fixed characteristics
db = pd.merge(undocs, codebook, on = ['year', 'countryID'])
db = pd.merge(db, wbank2, on = ['year', 'countryID'])
db = pd.merge(db, wbankdet, on = 'countryID')

In [9]:
# Rename columns 
db = db.rename(columns = {'Session':'session','Name of Person Speaking':'speaker','text':'statement',
                          'Language':'language','Notes':'notes','indicator':'indic_id',
                          'Indicator Name':'indic_name','Country Name':'country_name','indicator_val':'indic_val',
                          'Post':'speaker_post'})

In [10]:
# Select features of interest
db = db[['session','year','countryID','country_name','region','statement','speaker','speaker_post','indic_id','indic_name','indic_val','income_level','language']]

In [12]:
# Drop duplicates and select features of interest
db_a = db.drop_duplicates(subset = ['year', 'countryID'])
db_a = db_a[['session','year','countryID','country_name','region','statement','speaker','speaker_post','income_level','language']]

In [14]:
# Reshape to guarantee row driver: country-year
db_b = db.pivot_table(index = ['year','countryID'], columns = 'indic_name', values = 'indic_val', dropna = False)

In [15]:
# Merge with World Bank indicators again
final_df = pd.merge(db_b, db_a , on = ['year', 'countryID'])

In [16]:
# Export data to .txt format 
db.to_csv('final_df.csv')